# 임베딩(Embedding) 첫걸음 노트북

토크나이저 다음 단계로, "토큰을 숫자 벡터로 표현"하는 과정을 아주 기초부터 체험하는 노트입니다.
- Step 0: 오늘 사용할 문장과 토큰 목록 정의
- Step 1: 토큰을 숫자 ID로 바꾸는 "사전" 만들기
- Step 2: One-hot 벡터로 표현해 보기
- Step 3: Bag-of-Words(빈도 벡터) 살펴보기
- Step 4: 임의의 임베딩 테이블을 만들어 벡터를 꺼내 보기
- Step 5: (선택) 문장 전체를 하나의 벡터로 요약하는 간단한 방법


## Step 0 · 실험용 문장과 토큰 목록
- 토크나이저에서 사용했던 문장을 그대로 씁니다.
- 토큰은 구두점 인지 토크나이저 결과 중 자주 나오는 것만 추려서 사용합니다.


In [1]:
sentences = [
    "저의 이름 배승도에요.",
    "전화 번호는 010-1234-5678이에요.",
    "Dobby is Free!"
]

vocab_tokens = [
    "[PAD]", "[UNK]", "저의", "이름", "배승도", "전화", "번호는", "010", "-", "1234", "5678", "이에요", "Dobby", "is", "Free", "!"
]

sentences, vocab_tokens


(['저의 이름 배승도에요.', '전화 번호는 010-1234-5678이에요.', 'Dobby is Free!'],
 ['[PAD]',
  '[UNK]',
  '저의',
  '이름',
  '배승도',
  '전화',
  '번호는',
  '010',
  '-',
  '1234',
  '5678',
  '이에요',
  'Dobby',
  'is',
  'Free',
  '!'])

## Step 1 · 토큰 → 숫자 ID 매핑
- 토크나이저가 만들어 놓은 vocab을 직접 흉내 내 봅니다.
- 사전을 만들고, 토큰이 없으면 `[UNK]` ID를 사용합니다.


In [2]:
token_to_id = {token: idx for idx, token in enumerate(vocab_tokens)}
id_to_token = {idx: token for token, idx in token_to_id.items()}

print("토큰→ID 예시:")
for token in ["저의", "번호는", "없는토큰"]:
    print(token, "→", token_to_id.get(token, token_to_id["[UNK]"]))


토큰→ID 예시:
저의 → 2
번호는 → 6
없는토큰 → 1


## Step 2 · One-hot 벡터 만들기
- vocab 크기만큼 길이를 가진 벡터를 만들고, 해당 토큰 위치만 1로 표시합니다.
- 가장 단순하지만, 희소(Sparse)하고 길이가 길다는 단점이 있습니다.


In [3]:
import numpy as np

vocab_size = len(vocab_tokens)

def one_hot(token: str) -> np.ndarray:
    vec = np.zeros(vocab_size, dtype=int)
    idx = token_to_id.get(token, token_to_id["[UNK]"])
    vec[idx] = 1
    return vec

sample_vector = one_hot("전화")
print("'전화' one-hot 벡터 (앞부분만):", sample_vector[:12])


'전화' one-hot 벡터 (앞부분만): [0 0 0 0 0 1 0 0 0 0 0 0]


## Step 3 · Bag-of-Words (빈도 벡터)
- 문장 전체를 하나의 벡터로 표현하는 첫걸음.
- 등장 횟수를 세어 벡터에 기록합니다 (순서는 무시).
- 단점: 단어 순서를 잃고, 길이가 여전히 큽니다.


In [4]:
from collections import Counter


def bag_of_words(sentence: str) -> np.ndarray:
    tokens = sentence.replace(".", "").replace("!", "").split()
    counter = Counter(tokens)
    vec = np.zeros(vocab_size, dtype=int)
    for token, count in counter.items():
        idx = token_to_id.get(token, token_to_id["[UNK]"])
        vec[idx] = count
    return vec

for s in sentences:
    print(s)
    print("→ BOW 벡터(앞부분):", bag_of_words(s)[:12])
    print()


저의 이름 배승도에요.
→ BOW 벡터(앞부분): [0 1 1 1 0 0 0 0 0 0 0 0]

전화 번호는 010-1234-5678이에요.
→ BOW 벡터(앞부분): [0 1 0 0 0 1 1 0 0 0 0 0]

Dobby is Free!
→ BOW 벡터(앞부분): [0 0 0 0 0 0 0 0 0 0 0 0]



## Step 4 · 임의의 임베딩 테이블 흉내
- 실제 모델은 학습을 통해 각 토큰에 짧은 실수 벡터(예: 768차원)를 부여합니다.
- 여기서는 이해를 돕기 위해 3차원짜리 작은 테이블을 직접 만들어 봅니다.


In [ ]:
np.random.seed(42)
embedding_dim = 3

embedding_table = {
    token: np.round(np.random.uniform(-1, 1, embedding_dim), 3)
    for token in vocab_tokens
}

for token in ["저의", "전화", "Free", "없는토큰"]:
    emb = embedding_table.get(token, np.zeros(embedding_dim))
    print(token.ljust(6), "→", emb)


## Step 5 · 문장 벡터 간단히 만들기(선택)
- 여러 토큰 벡터를 평균 내면 “문장의 대략적인 의미 벡터”가 됩니다.
- 실제 모델도 비슷한 아이디어를 더 정교하게 사용합니다.
- 여기서는 Step 4의 임베딩 테이블을 이용해 평균 벡터를 계산해 봅니다.


In [ ]:
def sentence_embedding(sentence: str) -> np.ndarray:
    tokens = sentence.replace(".", "").replace("!", "").split()
    vectors = []
    for token in tokens:
        vectors.append(embedding_table.get(token, np.zeros(embedding_dim)))
    if not vectors:
        return np.zeros(embedding_dim)
    return np.mean(vectors, axis=0)

for s in sentences:
    print(s)
    print("→ 문장 벡터:", np.round(sentence_embedding(s), 3))
    print()


### 마무리 메모
- 토큰을 숫자 ID로 바꾸면 모델이 “사전을 공유”할 수 있습니다.
- One-hot과 Bag-of-Words는 단순하지만 차원이 매우 큽니다.
- 임베딩 테이블은 각 토큰에 짧은 실수 벡터를 부여해 효율적으로 표현합니다.
- 문장 임베딩은 여러 토큰 벡터를 조합(평균 또는 가중합)해서 얻을 수 있습니다.
- 실제 모델은 이 과정을 학습으로 자동화하며, 차원도 훨씬 크고 정보가 풍부합니다.
